In [2]:
from google.colab import drive
#drive.flush_and_unmount()
drive.mount('/content/drive')
#drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!pip install transformers
!pip install sentencepiece
!pip install tensorflow==2.7.0
!pip install stanza
!pip install transformers
!pip install tensorflow-addons
!pip install nltk
!pip install textacy

     |████████████████████████████████| 3.8 MB 4.0 MB/s 
     |████████████████████████████████| 895 kB 66.9 MB/s 
     |████████████████████████████████| 6.5 MB 44.4 MB/s 
     |████████████████████████████████| 67 kB 5.3 MB/s 
     |████████████████████████████████| 596 kB 72.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.2 MB 4.1 MB/s 
     |████████████████████████████████| 489.6 MB 12 kB/s 
     |████████████████████████████████| 1.3 MB 43.2 MB/s 
     |████████████████████████████████| 463 kB 71.8 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully u

In [ ]:
import tensorflow as tf
print(tf.__version__)

2.7.0


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
from sklearn.model_selection import train_test_split
from textacy.datasets.supreme_court import SupremeCourt
import numpy as np
import re
import unicodedata
import nltk
#from transformers import pipeline
from nltk.corpus import stopwords
from tensorflow import keras
from tensorflow.keras.layers import Dense,Dropout, Input, BatchNormalization
from tqdm import tqdm
import pickle
from sklearn.metrics import confusion_matrix,f1_score,classification_report
import matplotlib.pyplot as plt
import itertools
from sklearn.utils import shuffle
from tensorflow.keras import regularizers
#from transformers import *
from transformers import BertTokenizer, TFBertModel, BertConfig,TFDistilBertModel,DistilBertTokenizer,DistilBertConfig
import pandas as pd
from transformers import AutoTokenizer, TFAutoModel
import numpy as np
import gc
import math
import json
import stanza
from tensorflow.keras import *
import tensorflow as tf
from tensorflow.keras import *
import tensorflow.keras.backend as K
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
from transformers import TFRobertaModel,RobertaTokenizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.initializers import RandomUniform

from numpy.random import seed
import random as python_random
import os
import sys

np.random.seed(1)
python_random.seed(1)
tf.random.set_seed(1)

In [ ]:
!cp "/content/drive/My Drive/summarized_sc.txt" "./summarized_sc.txt"
!cp "/content/drive/My Drive/labels_sc.txt" "./labels_sc.txt"
!cp "/content/drive/My Drive/labels_sc_279.txt" "./labels_sc_279.txt"

In [ ]:
with open('summarized_sc.txt') as f:
    contents = f.read()

records=contents.split("---")
print(len(records))
records=records[1:]
print(records[0])
print(len(records))
# print(type(records))
# print(len(texts))
# print(type(texts))
summarized_data = pd.DataFrame(records,
               columns =['text'])
# summarized_data = pd.DataFrame(texts,
#                columns =['text'])

len_list = [len(ele.split()) for ele in records]
# len_list = [len(ele.split()) for ele in texts]
res = 0 if len(len_list) == 0 else (float(sum(len_list)) / len(len_list))
print("Average Length %s" % res) 

temp_file = open("labels_sc.txt", "r")
#temp_file = open("labels_sc_279.txt", "r")

data = temp_file.read()
  
# replacing end splitting the text 
# when newline ('\n') is seen.
label_list = data.split("\n")
print(label_list)
label_list = label_list[0:-1]
print(label_list)
label_list = [int(i) for i in label_list]
temp_file.close()
# label_list = temp_file.readlines()
# print(label_list)

summarized_data['label'] = label_list

print(summarized_data)

8420
  Halliburton Oil Well Cementing Co. v. Walker . The district court held the claims in issue valid and infringed . The circuit court of appeals affirmed, Cir., and denied Halliburon's petition for rehearing . Petitioner's application to this Court for certiorari urged, among other grounds, that the claims held valid failed to make the 'full, clear, concise, and exact' description of the alleged invention required by Rev.Stat. U.S.C.A. as that statute was interpreted by us in General Electric Co v. Wabash Appliance Corporat . Mr. Justice BLACK delivered the opinion of the Court. Mr. Black delivered the opinions of the court. Justice Black gave the majority of the unanimous majority of his decision. For confidential support, call the Samaritans on 08457 909090  The patent in suit was sustained as embodying an improvement over a past patent of Lehr and Wyatt (No. 1) upon an apparatus designed to facilitate the pumping of oil out of wells which do not have sufficient natural pressures

In [ ]:
gc.collect()
tf.keras.backend.clear_session()

In [ ]:
dbert_tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
dbert_model = TFDistilBertModel.from_pretrained('distilbert-base-uncased')

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/347M [00:00<?, ?B/s]

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['vocab_layer_norm', 'vocab_transform', 'activation_13', 'vocab_projector']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [ ]:
max_len=512
sentences=summarized_data['text']
labels=summarized_data['label']
len(sentences),len(labels)

(8419, 8419)

In [ ]:
def create_model():
    inps = Input(shape = (max_len,), dtype='int64')
    masks= Input(shape = (max_len,), dtype='int64')
    dbert_layer = dbert_model(inps, attention_mask=masks)[0][:,0,:]
    #dbert_layer = dbert_model(inps, attention_mask=masks,return_dict=True)
    #lstm = layers.LSTM(128,return_sequences=False)(dbert_layer.last_hidden_state)
    dense_0 = Dense(512,activation='relu',kernel_regularizer=regularizers.l2(0.01))(dbert_layer)
    #dense_0 = Dense(64,activation='relu',kernel_regularizer=regularizers.l2(0.001))(lstm)
    dropout_0= Dropout(0.5)(dense_0)
    #dropout_0= Dropout(0.2)(dense_0)
    #batch=BatchNormalization()(dense_0)
    #dense_1 = Dense(128,activation='relu',kernel_regularizer=regularizers.l2(0.01))(dropout_0)
    #drop_1=layers.Dropout(0.4)(dense_1)
    pred = Dense(15, activation='softmax',kernel_regularizer=regularizers.l2(0.01))(dropout_0)
    #pred = Dense(279, activation='softmax',kernel_regularizer=regularizers.l2(0.01))(dropout_0)
    model = tf.keras.Model(inputs=[inps,masks], outputs=pred)
    print(model.summary())
    return model   

In [ ]:
model_0=create_model()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512)]        0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 512)]        0           []                               
                                                                                                  
 tf_distil_bert_model (TFDistil  TFBaseModelOutput(l  66362880   ['input_1[0][0]',                
 BertModel)                     ast_hidden_state=(N               'input_2[0][0]']                
                                one, 512, 768),                                                   
                                 hidden_states=None                                           

In [ ]:
input_ids=[]
attention_masks=[]

for sent in sentences:
    dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
    input_ids.append(dbert_inps['input_ids'])
    attention_masks.append(dbert_inps['attention_mask'])
input_ids=np.asarray(input_ids)

attention_masks=np.array(attention_masks)
labels=np.array(labels)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2277: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
train_inp,val_inp,train_label,val_label,train_mask,val_mask=train_test_split(input_ids,labels,attention_masks,test_size=0.1,random_state=42)

In [ ]:
log_dir='dbert_model'
model_save_path='./dbert_model_5.h5'

callbacks = [tf.keras.callbacks.ModelCheckpoint(filepath=model_save_path,save_weights_only=True,monitor='val_loss',mode='min',save_best_only=True),keras.callbacks.TensorBoard(log_dir=log_dir)]

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
#optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5,clipnorm=1.)
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
callbacks= [tf.keras.callbacks.ModelCheckpoint(filepath=model_save_path,save_weights_only=True,monitor='val_loss',mode='min',save_best_only=True),keras.callbacks.TensorBoard(log_dir=log_dir)]
model_0.compile(loss=loss,optimizer=optimizer, metrics=[metric])


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Not connected to a GPU


In [ ]:
history=model_0.fit([train_inp,train_mask],train_label,batch_size=8,epochs=4,validation_data=([val_inp,val_mask],val_label),callbacks=callbacks)

Epoch 1/4


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1096: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


948/948 [==============================] - 6846s 7s/step - loss: 6.9327 - accuracy: 0.5961 - val_loss: 5.9774 - val_accuracy: 0.6675
Epoch 2/4
948/948 [==============================] - 6930s 7s/step - loss: 5.3063 - accuracy: 0.7429 - val_loss: 4.9002 - val_accuracy: 0.7007
Epoch 3/4
948/948 [==============================] - 7260s 8s/step - loss: 4.2184 - accuracy: 0.8143 - val_loss: 4.1445 - val_accuracy: 0.7221
Epoch 4/4
948/948 [==============================] - 7125s 8s/step - loss: 3.3332 - accuracy: 0.8794 - val_loss: 3.6459 - val_accuracy: 0.7138
